### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected Remain 204"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_Remain_24"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [7]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [10]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [11]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [12]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [13]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,2139.412,2143.488,aysun sen buradasın değil mi,yK7iNLl2Tqo
1,5384.942,5385.944,canı acıyacak,aGQDt0mDyCU
2,6551.040,6552.500,senin paran benim param selim,o5Zsr5b9DYk
3,251.380,255.409,hem şapka çizmek istedim hem de sayfayı renkle...,b9J6Ijxsiw4
4,545.079,548.029,canım ailem için hediye almak istemiştim,GzF8RMV8Fss
...,...,...,...,...
3934202,1733.470,1736.620,bu mehmet ekipleri duruyor kanka bu ya,eypQZvq_52I
3934203,6005.000,6006.255,kapa çeneni,G7eFHY90kFs
3934204,2172.422,2172.972,ama bak,LVPw3VkNyME
3934205,3300.800,3302.720,senin de düzeltebileceğin bir durum yok,Wx6WqWXx0lU


In [14]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,ama şimdi olmaz
1,asla olmaz demek
2,ben gerçekten üzgünüm
3,benimle geliyor musun
4,bu küçük kız
5,bunun için teşekkürler
6,bütün gece mi
7,evet sanırım onları
8,geri ver şunu
9,hadi al şunu


In [ ]:
# Option 
# Skip if it doesn't need
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [15]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [16]:
len(search_list)

24

In [17]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama dostum,574.920,577.320,hadi ama dostum bunu yapabilirsin,tDUGNaOSKkA
1,yoksa bizi mi,2155.301,2157.228,yoksa bizi mi öğrendi,aMWKWcdyYMo
2,yoksa bizi mi,5865.890,5868.170,gerçekten babanı tanıyor olabilir mi yoksa biz...,70daKdkwjeY
3,yoksa bizi mi,5985.325,5986.924,yoksa bizi mi dolandırıyorsun,TVy2hkF_G9c
4,benimle geliyor musun,6543.215,6544.881,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.902,18.722,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6075.580,6077.120,benimle geliyor musun candan,aWAp9pXi_DI
7,benimle geliyor musun,2185.052,2186.671,elif sen benimle geliyor musun,1OLV2bjYgA0
8,benimle geliyor musun,6837.595,6839.085,benimle geliyor musun,awA8QjEncEU
9,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw


In [18]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama dostum,574.920000,576.083636,hadi ama dostum bunu yapabilirsin,tDUGNaOSKkA
1,yoksa bizi mi,2155.301000,2156.585667,yoksa bizi mi öğrendi,aMWKWcdyYMo
2,yoksa bizi mi,5867.281186,5867.860847,gerçekten babanı tanıyor olabilir mi yoksa biz...,70daKdkwjeY
3,yoksa bizi mi,5985.325000,5986.096931,yoksa bizi mi dolandırıyorsun,TVy2hkF_G9c
4,benimle geliyor musun,6543.215000,6544.233111,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.902000,18.014222,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6075.580000,6076.790000,benimle geliyor musun candan,aWAp9pXi_DI
7,benimle geliyor musun,2185.483733,2186.671000,elif sen benimle geliyor musun,1OLV2bjYgA0
8,benimle geliyor musun,6837.595000,6839.085000,benimle geliyor musun,awA8QjEncEU
9,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama dostum,574.320000,576.683636,hadi ama dostum bunu yapabilirsin,tDUGNaOSKkA
1,yoksa bizi mi,2154.701000,2157.185667,yoksa bizi mi öğrendi,aMWKWcdyYMo
2,yoksa bizi mi,5866.681186,5868.460847,gerçekten babanı tanıyor olabilir mi yoksa biz...,70daKdkwjeY
3,yoksa bizi mi,5984.725000,5986.696931,yoksa bizi mi dolandırıyorsun,TVy2hkF_G9c
4,benimle geliyor musun,6542.615000,6544.833111,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.302000,18.614222,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6074.980000,6077.390000,benimle geliyor musun candan,aWAp9pXi_DI
7,benimle geliyor musun,2184.883733,2187.271000,elif sen benimle geliyor musun,1OLV2bjYgA0
8,benimle geliyor musun,6836.995000,6839.685000,benimle geliyor musun,awA8QjEncEU
9,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw


In [20]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,hadi ama dostum,574,577,hadi ama dostum bunu yapabilirsin,tDUGNaOSKkA
1,yoksa bizi mi,2155,2157,yoksa bizi mi öğrendi,aMWKWcdyYMo
2,yoksa bizi mi,5867,5868,gerçekten babanı tanıyor olabilir mi yoksa biz...,70daKdkwjeY
3,yoksa bizi mi,5985,5987,yoksa bizi mi dolandırıyorsun,TVy2hkF_G9c
4,benimle geliyor musun,6543,6545,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16,19,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6075,6077,benimle geliyor musun candan,aWAp9pXi_DI
7,benimle geliyor musun,2185,2187,elif sen benimle geliyor musun,1OLV2bjYgA0
8,benimle geliyor musun,6837,6840,benimle geliyor musun,awA8QjEncEU
9,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw


In [21]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hadi ama dostum,574,577,hadi ama dostum bunu yapabilirsin,tDUGNaOSKkA,https://www.youtube.com/watch?v=tDUGNaOSKkA&t=...
1,yoksa bizi mi,2155,2157,yoksa bizi mi öğrendi,aMWKWcdyYMo,https://www.youtube.com/watch?v=aMWKWcdyYMo&t=...
2,yoksa bizi mi,5867,5868,gerçekten babanı tanıyor olabilir mi yoksa biz...,70daKdkwjeY,https://www.youtube.com/watch?v=70daKdkwjeY&t=...
3,yoksa bizi mi,5985,5987,yoksa bizi mi dolandırıyorsun,TVy2hkF_G9c,https://www.youtube.com/watch?v=TVy2hkF_G9c&t=...
4,benimle geliyor musun,6543,6545,benimle geliyor musun gelmiyor musun,liz1ODAVzyk,https://www.youtube.com/watch?v=liz1ODAVzyk&t=...
5,benimle geliyor musun,16,19,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU,https://www.youtube.com/watch?v=sJ9F9o0JySU&t=16s
6,benimle geliyor musun,6075,6077,benimle geliyor musun candan,aWAp9pXi_DI,https://www.youtube.com/watch?v=aWAp9pXi_DI&t=...
7,benimle geliyor musun,2185,2187,elif sen benimle geliyor musun,1OLV2bjYgA0,https://www.youtube.com/watch?v=1OLV2bjYgA0&t=...
8,benimle geliyor musun,6837,6840,benimle geliyor musun,awA8QjEncEU,https://www.youtube.com/watch?v=awA8QjEncEU&t=...
9,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw,https://www.youtube.com/watch?v=Ri8OHAHmTJw&t=...


In [22]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

12

In [23]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [24]:
search_all_set.difference(search_result_set)

{'ben gerçekten üzgünüm',
 'bunun için teşekkürler',
 'evet sanırım onları',
 'haydi git buradan',
 'herkes kendi geliyor',
 'hey neler oluyor',
 'oh hadi ama',
 'onunla birlikte değil',
 'pekala hadi bakalım',
 'selam baba merhaba',
 'tabii eğer o',
 'şimdi tekrar söyle'}

In [25]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,şunu,2
2,yoksa,2
3,olmaz,2
4,gece,2
5,hadi,2
6,ama,2
1,mi,2
23,bütün,1
20,dostum,1
21,demek,1


In [ ]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [26]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [27]:
set_word.difference(set_count)

{'adam',
 'anne',
 'artık',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'bana',
 'başka',
 'belki',
 'ben',
 'bence',
 'beni',
 'benim',
 'bile',
 'biliyor',
 'biliyorsun',
 'biliyorum',
 'bilmiyorum',
 'bir',
 'biraz',
 'biri',
 'birkaç',
 'birlikte',
 'biz',
 'bize',
 'bizim',
 'bugün',
 'buna',
 'bunu',
 'bunun',
 'burada',
 'buraya',
 'böyle',
 'büyük',
 'da',
 'daha',
 'dakika',
 'devam',
 'değil',
 'değilim',
 'diye',
 'doğru',
 'dur',
 'ederim',
 'efendim',
 'en',
 'et',
 'evet',
 'eğer',
 'fazla',
 'gel',
 'geldi',
 'gerek',
 'gerçek',
 'gerçekten',
 'gibi',
 'git',
 'göre',
 'gün',
 'güzel',
 'hakkında',
 'hala',
 'harika',
 'haydi',
 'hayır',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiç',
 'hiçbir',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'için',
 'içinde',
 'iş',
 'işte',
 'kadar',
 'kendi',
 'ki',
 'kimse',
 'kötü',
 'lazım',
 'lütfen',
 'merhaba',
 'misin',
 'mu',
 'mü',
 'mısın',
 'nasıl',
 'ne',
 'neden',
 'neler',
 'nerede',

In [28]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [27]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_67_Threegram_Remain_Word_10_Youtube_0.6s_Timeshift_Result2.xlsx',
 'Turkish_10_Threegram_Remain_Word2_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 